In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from mpl_toolkits.mplot3d.art3d import Line3DCollection
%matplotlib qt

# Part 1
Some constants

In [ ]:
charge = -1.9e-10 # coulombs

initial_vx = 20000 # mm/s
initial_vy = 0 # mm/s
initial_vz = 0 # mm/s
initial_x = 0 # Position, MM
initial_y = 0 # Position, MM
initial_z = 107.95 # Position, MM
initial_t = 0 # Seconds

droplet_diameter = 84e-6 # Meteres
droplet_density = 1000 # kg/m^3
droplet_mass = ( (4/3) * (droplet_diameter/2) ** 3) * math.pi * droplet_density
print_resolution = 300 # dpi

plate_voltage = 0 # Volts, for part 1
plate_seperation = 0.001 # Meters
plate_start = 1.25 # Position MM
plate_length = 0.5 # MM
plate_end = plate_start + plate_length # Position, MM
paper_end = 3 # MM

def reset_parameters():
    global charge, initial_vx, initial_vy, initial_vz, initial_x, initial_y, initial_z, initial_t
    global droplet_diameter, droplet_density, droplet_mass, print_resolution
    global plate_voltage, plate_seperation, plate_start, plate_length, plate_end, paper_end
    charge = -1.9e-10 # coulombs

    initial_vx = 20000 # mm/s
    initial_vy = 0 # mm/s
    initial_vz = 0 # mm/s
    initial_x = 0 # Position, MM
    initial_y = 0 # Position, MM
    initial_z = 107.95 # Position, MM
    initial_t = 0 # Seconds

    droplet_diameter = 84e-6 # Meteres
    droplet_density = 1000 # kg/m^3
    droplet_mass = ( (4/3) * (droplet_diameter/2) ** 3) * math.pi * droplet_density
    print_resolution = 300 # dpi

    plate_voltage = 0 # Volts, for part 1
    plate_seperation = 0.001 # Meters
    plate_start = 1.25 # Position MM
    plate_length = 0.5 # MM
    plate_end = plate_start + plate_length # Position, MM
    paper_end = 3 # MM


## Calculations For Question 1
We are assuming no voltage is applied to the plates. <br>
We are looking for how long it takes for the droplet to reach the paper. <br>
Due to the droplet being so small, we can negate air resistance. Therefore, the velocity will remain constant for X, Y and Z, for this case. <br>
While we have declared a part for the Y and Z component, we will not use it in this case, as there will be zero force applied and zero position changed in the Y and Z direction (for question 1)<br>

In [ ]:
reset_parameters()

droplet = ({'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]})

electric_field = plate_voltage / plate_seperation # Should be zero in this case, due to no plate voltage

step = 0
time_step = 1e-6

while (droplet['position_x'][step] <= paper_end):
    droplet['time'].append( time_step * (step + 1) )

    if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
        force_z = 0
    else: # If it is between the plates
        force_z = electric_field * charge # This should be zero in this case, due to there being no electric field between the plates

    droplet['velocity_x'].append(initial_vx) # Should not vary here
    droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
    droplet['velocity_y'].append(initial_vy)

    droplet['position_z'].append( (droplet['velocity_z'][step] * time_step) + droplet['position_z'][step] )
    droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
    droplet['position_y'].append(0) # It is known it will not change in this scenario
    
    step = step + 1


## Plotting Question 1

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 1, initial_z + 1, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet Trajectory, when V=0 to draw a single dot')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(np.min(droplet['position_y']) - 1, np.max(droplet['position_y']) + 1)
ax1.set_xlim(np.min(droplet['position_x']), np.max(droplet['position_x']) + 0.1)
ax1.set_zlim(np.min(droplet['position_z']) - 1, np.max(droplet['position_z']) + 1)
ax1.view_init(elev=5, azim=-105)

dot1 = ax1.scatter([], [], [], color='r', label='Position')

def update_position(frame):
    dot1._offsets3d = (droplet['position_x'][frame:frame+1], droplet['position_y'][frame:frame+1], droplet['position_z'][frame:frame+1])
    return dot1,

ani1 = animation.FuncAnimation(fig1, update_position, frames=step, interval=10, blit=False, repeat=False)



# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question1.mp4", writer='ffmpeg', fps=480)

# Calculating time it took to to reach the paper
print(f"The time it took to reach the paper was {droplet['time'][-1]} seconds")


# Question 2
Simulate the act of drawing a vertical line (similar to the letter “I” ) along the center of the paper at 300 dpi resolution. How much time would it take to draw the letter ‘I’? <br>
In this case, the voltage needs to be dynamically varied accross each step. <br>
This will affect the Z position, which will be calcualted for each dot trajectory <br>
We can approximate that if we are require 300dpi, this equated to about 12 dpmm (dots per milimeter). This tells us how many dots we need to draw per milimeter <br>. 

In [ ]:
reset_parameters()

plate_voltage = 2330.5 
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 79


# Plotting Question 2

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 2.75, initial_z + 2.75, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet trajectories varying voltage to draw the letter "I"')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, 3.1)
ax1.set_zlim(initial_z - 2.75, initial_z + 2.75)
ax1.view_init(elev=5, azim=-105)
dots = []
for i, droplet in enumerate(droplets):
    dot = ax1.scatter([], [], [], color='r', label=f'Droplet {i+1}' if i == 0 else "")
    dots.append(dot)

def update_position(frame):
    current_droplet = frame // step 
    current_frame = frame % step
    
    if current_droplet < len(droplets):
        droplet = droplets[current_droplet]
        dots[current_droplet]._offsets3d = (droplet['position_x'][current_frame:current_frame+1],  droplet['position_y'][current_frame:current_frame+1], droplet['position_z'][current_frame:current_frame+1])
    return dots
total_frames = len(droplets) * step
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=False, repeat=False)


# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question2.mp4", writer='ffmpeg', fps=480)


print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1], "seconds")


# Question 3
We are now looking to draw the 'I' as big as possible. This was nearly achieved in the previous section, but it can still be larger. <br>
We are also going to speed up the process, by having it so when a droplet exits the parallel plates, the next droplet will enter. <br>

In [ ]:
reset_parameters()

plate_voltage = 2360 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 80


## Plotting Question 3


In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 2.8, initial_z + 2.8, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Droplet trajectories when the voltage is to draw the letter "I", faster')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, 3.1)
ax1.set_zlim(initial_z - 2.75, initial_z + 2.75)
ax1.view_init(elev=5, azim=-105)
dots = []
for i, droplet in enumerate(droplets):
    dot = ax1.scatter([], [], [], color='r', label=f'Droplet {i+1}' if i == 0 else "")
    dots.append(dot)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r')  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=True, repeat=False)

# The time will be halved
print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)


# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question3.mp4", writer='ffmpeg', fps=480)

# Question 4: 
## Resimulate the act of drawing letter ‘I’ as big as possible consuming as small time as possible for each of the following adjustment of the parameter:
### A. The distance between the capacitor and the paper L2 is threefold increased, and everything else remains same 

In [ ]:
reset_parameters()

# Increasing the distance between the capacitor and paper 3x
paper_end = 3 + 2.5

plate_voltage = 2360 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 7) - (initial_z - 7) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 28


## Plotting the results of question 4A

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 15, initial_z + 15, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Drawing the letter "I" with increased distance to paper')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, paper_end + 0.1)
ax1.set_zlim(initial_z - 10, initial_z + 10)
ax1.view_init(elev=5, azim=-105)
dots = []
for i, droplet in enumerate(droplets):
    dot = ax1.scatter([], [], [], color='r', label=f'Droplet {i+1}' if i == 0 else "", s=5)
    dots.append(dot)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 4

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r')  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=True, repeat=False)

print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)


# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question4a.mp4", writer='ffmpeg', fps=480)

### B. L1 is twofold increased, and everything else remains same

In [ ]:
reset_parameters()
# Increasing the distance between the capacitor and cannon 2x
paper_end = 3 + 1.25
plate_start = 1.25 + 1.25
plate_end = plate_start + plate_length

plate_voltage = 2360 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
print(num_droplets)
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 80


## Plotting the results

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 5, initial_z + 5, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Drawing the letter "I" with increased distance to capacitor')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, paper_end + 0.1)
ax1.set_zlim(initial_z - 5.1, initial_z + 5.1)
ax1.view_init(elev=5, azim=-105)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r')  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=True, repeat=False)

print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)

# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question4b.mp4", writer='ffmpeg', fps=480)

### C. Droplet diameter is tenfold increased, and everything else remains same

In [ ]:
reset_parameters()

# Setting droplet diameter to be 10x larger, adjusting the mass accordingly
droplet_diameter = 840e-6 # Meters
droplet_mass = ( (4/3) * (droplet_diameter/2) ** 3) * math.pi * droplet_density

plate_voltage = 1000000 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12  # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) ) / 10) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 333333.333333


## Plotting the results

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 5, initial_z + 5, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Drawing the letter "I" with 10x larger droplets')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, paper_end + 0.1)
ax1.set_zlim(initial_z - 5.1, initial_z + 5.1)
ax1.view_init(elev=5, azim=-105)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r', markersize=10)  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=False, repeat=False)

print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)

# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question4c.mp4", writer='ffmpeg', fps=480)

### D. The horizontal speed at which the gun shoots the droplet is twofold increased, and everything else remains same

In [ ]:
reset_parameters()
# Setting droplet horizontal velocity to be 2x faster
initial_vx = 20000 * 2 # MM/second

plate_voltage = 2360 * 5 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 80 * 5


## Plotting the results

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 5, initial_z + 5, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Drawing the letter "I" with 2x faster droplets')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, paper_end + 0.1)
ax1.set_zlim(initial_z - 5.1, initial_z + 5.1)
ax1.view_init(elev=5, azim=-105)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r')  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=True, repeat=False)
print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)

# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question4d.mp4", writer='ffmpeg', fps=480)

### E. The charge of the droplet is fivefold increased, and everything else remains same.

In [ ]:
reset_parameters()
# Increasing the charge of the droplet to 5x
charge = -1.9e-10 * 5 # Coulombs

plate_voltage = 2360 / 5 # This will by dynamically changing to draw an 'I'
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage': [plate_voltage], 'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
num_droplets = math.ceil(dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
print(num_droplets)
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [initial_vy], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    electric_field = plate_voltage / plate_seperation # V/MM
    while (droplet['position_x'][step] <= paper_end):
        voltage_over_time['voltage'].append(plate_voltage)
        voltage_over_time['time'].append(time_step * (total_steps + 1))
        droplet['time'].append( time_step * (step + 1) )

        if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
            force_z = 0
        else: # If it is between the plates
            force_z = electric_field * charge
        droplet['velocity_x'].append(initial_vx) # Should not vary here
        droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
        droplet['velocity_y'].append(initial_vy)

        droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
        droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
        droplet['position_y'].append(0) # It is known it will not change in this scenario
        
        step = step + 1
        total_steps = total_steps + 1
    plate_voltage = plate_voltage - 80 / 5


## Plotting the results

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-0.2, 0.2, 10)
X, Y = np.meshgrid(x, y)
# Creates first plate
Z = np.full_like(X, initial_z + 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Creates second plate
Z = np.full_like(X, initial_z - 0.5)  
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)

# Similiar to before, but for the paper
y = np.linspace(-1, 1, 10)
z = np.linspace(initial_z - 5, initial_z + 5, 10)
Y, Z = np.meshgrid(y, z)
# Plot the paper
X = np.full_like(Y, paper_end) 
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)


ax1.set_title('Drawing the letter "I" with 5x charge droplets')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(-1, 1)
ax1.set_xlim(0, paper_end + 0.1)
ax1.set_zlim(initial_z - 5.1, initial_z + 5.1)
ax1.view_init(elev=5, azim=-105)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r')  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=True, repeat=False)

print("The time it took to draw the 'I', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1] / 2, "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'I'")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage']) - 10 , np.max(voltage_over_time['voltage']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)

# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question4e.mp4", writer='ffmpeg', fps=480)

# Question 5
If you want to draw the letter ‘H’ on the paper, you have to add an additional capacitor to impose a velocity component to the droplet. Design the additional capacitor to draw the biggest possible ‘H’ on the paper.  You have to decide the design parameter of the capacitors to draw the ‘H’ shape (length, width of the capacitor plates, and their spacing). Simulate the entire process of drawing the biggest possible ‘H’ shape in 300 dpi resolution. Plot the voltage profile V(t) applied across the capacitor vs time t for both the capacitor. Both the voltage profile would look like a staircase. <br>

For this case, I will choose a pair of parallel plates that are exactly the same as the previous pair, but on a different axis. This way, all the letters can be square and symmetrical. <br>
Because of this, there will now be a velocity on the y-axis. <br>

In [ ]:
reset_parameters()

plate_voltage_z = 2360 # Initialize the z voltage
plate_voltage_y = 2360 # Initialize the y voltage as well
droplets = [] # Keeps a array of droplets and their data
voltage_over_time = {'voltage_z': [plate_voltage_z], 'voltage_y': [plate_voltage_y],  'time': [0]} # This will keep track of the voltages that were applied to each droplet

dpmm = 12 # Roughly, dots per milimeter
# The number of dots needed will be triple, as we will essentially be drawing 3 lines, all the same length
num_droplets = math.ceil(3 * dpmm *  ( (initial_z + 2.5) - (initial_z - 2.5) )) # number of dots needed 
for i in range(num_droplets):
    droplet = {'position_x': [initial_x], 'position_y': [initial_y], 'position_z': [initial_z],
                'velocity_x': [initial_vx], 'velocity_y': [0], 'velocity_z': [0],
                'time': [initial_t]}
    droplets.append(droplet)

time_step = 1e-6
total_steps = 0
for i, droplet in enumerate(droplets):
    step = 0
    # Draw the first vertical line, on the left (from our perspective)
    # We keep the voltage varying on the z axis, and constant on the y axis to the limit
    if (i < math.ceil(num_droplets * (1/3))):
        electric_field_z = plate_voltage_z / plate_seperation # V/MM
        electric_field_y = plate_voltage_y / plate_seperation # V/MM
        while (droplet['position_x'][step] <= paper_end):
            voltage_over_time['voltage_z'].append(plate_voltage_z)
            voltage_over_time['voltage_y'].append(plate_voltage_y)
            voltage_over_time['time'].append(time_step * (total_steps + 1))
            droplet['time'].append( time_step * (step + 1) )

            if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
                force_z = 0
                force_y = 0
            else: # If it is between the plates
                force_z = electric_field_z * charge
                force_y = electric_field_y * charge
            droplet['velocity_x'].append(initial_vx) # Should not vary here
            droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
            droplet['velocity_y'].append( ( (force_y * time_step) / droplet_mass) +  droplet['velocity_y'][step])

            droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
            droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
            droplet['position_y'].append( (droplet['velocity_y'][step] * time_step * 1000) + droplet['position_y'][step])
            step = step + 1
            total_steps = total_steps + 1
        plate_voltage_z = plate_voltage_z - 80
        continue

    # Draw the second vertical line, on the left (from our perspective)
    if (i < math.ceil(num_droplets * (2/3))):
        electric_field_z = (plate_voltage_z + 4720) / plate_seperation # V/MM
        electric_field_y = (plate_voltage_y - 4720) / plate_seperation # V/MM
        while (droplet['position_x'][step] <= paper_end):
            voltage_over_time['voltage_z'].append(plate_voltage_z + 4720)
            voltage_over_time['voltage_y'].append(plate_voltage_y - 4720)
            voltage_over_time['time'].append(time_step * (total_steps + 1))
            droplet['time'].append( time_step * (step + 1) )

            if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
                force_z = 0
                force_y = 0
            else: # If it is between the plates
                force_z = electric_field_z * charge
                force_y = electric_field_y * charge
            droplet['velocity_x'].append(initial_vx) # Should not vary here
            droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
            droplet['velocity_y'].append( ( (force_y * time_step) / droplet_mass) +  droplet['velocity_y'][step])

            droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
            droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
            droplet['position_y'].append( (droplet['velocity_y'][step] * time_step * 1000) + droplet['position_y'][step])
            step = step + 1
            total_steps = total_steps + 1
        plate_voltage_z = plate_voltage_z - 80
        continue
    
    # Draw the third line, which is horizontal
    if(i < num_droplets):
        electric_field_z = 0 # V/MM
        electric_field_y = plate_voltage_y / plate_seperation # V/MM
        while (droplet['position_x'][step] <= paper_end):
            voltage_over_time['voltage_z'].append(0)
            voltage_over_time['voltage_y'].append(plate_voltage_y)
            voltage_over_time['time'].append(time_step * (total_steps + 1))
            droplet['time'].append( time_step * (step + 1) )

            if (droplet['position_x'][step] < plate_start or droplet['position_x'][step] > plate_end): # Checks if the droplet is within the plates
                force_z = 0
                force_y = 0
            else: # If it is between the plates
                force_z = electric_field_z * charge
                force_y = electric_field_y * charge
            droplet['velocity_x'].append(initial_vx) # Should not vary here
            droplet['velocity_z'].append( ( (force_z * time_step) / droplet_mass) +  droplet['velocity_z'][step] )
            droplet['velocity_y'].append( ( (force_y * time_step) / droplet_mass) +  droplet['velocity_y'][step])

            droplet['position_z'].append( (droplet['velocity_z'][step] * time_step * 1000) + droplet['position_z'][step])
            droplet['position_x'].append( (initial_vx * time_step) + droplet['position_x'][step] )
            droplet['position_y'].append( (droplet['velocity_y'][step] * time_step * 1000) + droplet['position_y'][step])
            step = step + 1
            total_steps = total_steps + 1
        plate_voltage_y = plate_voltage_y - 80
        continue


## Plotting Question 5

In [ ]:
fig1 = plt.figure(figsize=(8, 6))
ax1 = fig1.add_subplot(111, projection='3d')

# Generate static parts once (plates, paper)
x = np.linspace(plate_start, plate_end, 10)
y = np.linspace(-1, 1, 10)
X, Y = np.meshgrid(x, y)

# Creates both plates for the z axis
Z = np.full_like(X, initial_z + 0.5)
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)
Z = np.full_like(X, initial_z - 0.5)
ax1.plot_surface(X, Y, Z, color='cyan', alpha=0.2)


x = np.linspace(1.25, 1.75, 10)
z = np.linspace(107.45, 108.45, 10)
X, Z = np.meshgrid(x, z)

# Creates both plates for the y axis
Y = np.full_like(X, initial_y + 1)
ax1.plot_surface(X, Y, Z, color='blue', alpha=0.2)
Y = np.full_like(X, initial_y - 1)
ax1.plot_surface(X, Y, Z, color='blue', alpha=0.2)

y = np.linspace(initial_y - 3, initial_y + 3, 10)
z = np.linspace(initial_z - 3, initial_z + 3, 10)
Y, Z = np.meshgrid(y, z)
# Creates the paper
X = np.full_like(Y, paper_end)
ax1.plot_surface(X, Y, Z, color='white', alpha=0.5)

# Set plot properties
ax1.set_title('Drawing the lettter "H" with 2 pairs of parallel plates.')
ax1.set_xlabel('X Position (MM)')
ax1.set_zlabel('Z Position (MM)')
ax1.set_ylabel('Y Position (MM)')
ax1.set_ylim(initial_y - 3.75, initial_y + 3.75)
ax1.set_xlim(0, 3.1)
ax1.set_zlim(initial_z - 3, initial_z + 3)
ax1.view_init(elev=5, azim=-110)

# Define the step for the droplets' animation
step = len(droplets[0]['position_x'])
after_plates = step // 2

# Compute start frames for each droplet
start_frames = [0]
for i in range(1, len(droplets)):
    start_frames.append(start_frames[-1] + after_plates)

# Total frames for the animation
total_frames = start_frames[-1] + step

lines = []
for i, droplet in enumerate(droplets):
    line, = ax1.plot([], [], [], marker='o', color='r')  # Placeholder for lines
    lines.append(line)

# Update function for the animation
def update_position(frame):
    for i, line in enumerate(lines):
        sf = start_frames[i]
        if frame < sf:
            line.set_data([], [])
            line.set_3d_properties([], 'z')
        elif frame < sf + step:
            lf = frame - sf
            line.set_data(droplets[i]['position_x'][lf:lf+1], droplets[i]['position_y'][lf:lf+1])
            line.set_3d_properties(droplets[i]['position_z'][lf:lf+1])
        else:
            line.set_data(droplets[i]['position_x'][-1:], droplets[i]['position_y'][-1:])
            line.set_3d_properties(droplets[i]['position_z'][-1:])
    return lines

# Animation
ani1 = animation.FuncAnimation(fig1, update_position, frames=total_frames, interval=1, blit=True, repeat=False)

print("The time it took to draw the 'H', which was", -droplets[0]['position_z'][-1] + droplets[num_droplets - 1]['position_z'][-1],"mm long, took,", voltage_over_time['time'][-1], "seconds")

fig2 = plt.figure(figsize=(8, 6))
ax2 = fig2.add_subplot(111)
ax2.set_title("Voltage vs. Time when drawing the letter 'H', Z axis plates")
ax2.set_ylabel('Voltage (V)')
ax2.set_xlabel('Time (s)')
ax2.set_ylim(np.min(voltage_over_time['voltage_z']) - 10 , np.max(voltage_over_time['voltage_z']) + 10)
ax2.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot2 = ax2.scatter([], [], color='b', label='voltage')
line2, = ax2.plot(voltage_over_time['time'], voltage_over_time['voltage_z'], 'b-')
def update_voltage(frame):
    dot2.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage_z'][frame:frame+1]])
    line2.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage_z'][frame:frame+1])
    line2.set_xdata()
    return dot2, line2
ani2 = animation.FuncAnimation(fig2, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)

fig3 = plt.figure(figsize=(8, 6))
ax3 = fig3.add_subplot(111)
ax3.set_title("Voltage vs. Time when drawing the letter 'H', Y axis plates")
ax3.set_ylabel('Voltage (V)')
ax3.set_xlabel('Time (s)')
ax3.set_ylim(np.min(voltage_over_time['voltage_y']) - 10 , np.max(voltage_over_time['voltage_y']) + 10)
ax3.set_xlim(np.min(voltage_over_time['time']), np.max(voltage_over_time['time']))

dot3 = ax3.scatter([], [], color='b', label='voltage')
line3, = ax3.plot(voltage_over_time['time'], voltage_over_time['voltage_y'], 'b-')
def update_voltage(frame):
    dot3.set_offsets(np.c_[voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage_y'][frame:frame+1]])
    line3.set_ydata(voltage_over_time['time'][frame:frame+1], voltage_over_time['voltage_y'][frame:frame+1])
    line3.set_xdata()
    return dot3, line3
ani3 = animation.FuncAnimation(fig3, update_voltage, frames=len(voltage_over_time['time']), interval=10, blit=False, repeat=False)


# To show the animation live, uncomment the line below
plt.show()
# Save animation to file (commoent out if you are viewing live)
# ani1.save("question5.mp4", writer='ffmpeg', fps=480)